In [1]:
import pandas as pd
import numpy as np
import joblib
import os
import sys

In [2]:
sys.path.append(os.path.join('.', "../../cbr_analyser/amr/"))

In [3]:
train_data = joblib.load("../masked_sentences_with_AMR_container_objects_train.joblib")

In [4]:
train_to_label = {}
for obj in train_data:
    train_to_label[obj[1].sentence.strip()] = obj[2]

In [5]:
base_predictions = pd.read_csv("basic_model/test.csv")

base_predictions.rename(columns={'predictions': 'base_predictions'}, inplace=True)
base_predictions.drop(columns=['true_labels', 'masked_articles'], inplace=True)

gcn_predictions = pd.read_csv("gcn/test.csv")
gcn_predictions.dropna(inplace=True)
gcn_predictions['cbr_label'] = list(map(lambda x: train_to_label[x], gcn_predictions['cbr']))
gcn_predictions = pd.concat([base_predictions, gcn_predictions], axis = 1)

simcse_predictions = pd.read_csv("simcse/test.csv")
simcse_predictions['cbr_label'] = list(map(lambda x: train_to_label[x], simcse_predictions['cbr']))
simcse_predictions = pd.concat([base_predictions, simcse_predictions], axis = 1)

empathy_predictions = pd.read_csv("empathy/test.csv")
empathy_predictions['cbr_label'] = list(map(lambda x: train_to_label[x], empathy_predictions['cbr']))
empathy_predictions = pd.concat([base_predictions, empathy_predictions], axis = 1)

In [7]:
for df, name in zip([simcse_predictions, gcn_predictions, empathy_predictions], ['simcse', 'gcn', 'empathy']):
    print(name)
    print('Number of cases that are actually the true labels', df[df['cbr_label'] == df['true_labels']].shape[0] / df.shape[0])

    helpful = df[(df['base_predictions'] != df['true_labels']) & (df['predictions'] == df['true_labels'])]
    harmful = df[(df['base_predictions'] == df['true_labels']) & (df['predictions'] != df['true_labels'])]
    
    
    print('number of helpful and harmful cases')
    print(len(helpful))
    print(len(harmful))
    
    print('Classes that they are helpful in')
    print(helpful['true_labels'].value_counts())
    
    print('$$$' * 10)
    
    print('classes that are harmful for')
    print(harmful['true_labels'].value_counts())
    
    print('***' * 10)

simcse
Number of cases that are actually the true labels 0.4101694915254237
number of helpful and harmful cases
24
28
Classes that they are helpful in
appeal to emotion         6
fallacy of credibility    4
fallacy of relevance      4
ad populum                4
circular reasoning        2
faulty generalization     2
intentional               1
false causality           1
Name: true_labels, dtype: int64
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
classes that are harmful for
fallacy of logic          5
ad hominem                5
fallacy of relevance      3
faulty generalization     3
circular reasoning        2
fallacy of credibility    2
appeal to emotion         2
fallacy of extension      2
ad populum                2
intentional               1
false causality           1
Name: true_labels, dtype: int64
******************************
gcn
Number of cases that are actually the true labels 0.3423728813559322
number of helpful and harmful cases
19
63
Classes that they are helpful in
faulty general